<a href="https://colab.research.google.com/github/weipengdeng/topic_modelling/blob/main/TopicModel_Future_Trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gdown

# Your Google Drive link
file_id = '1O2StrVyMCiv-fwheMncKhVyfW7mWcDB3'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file first
output = 'Master_Analysis.csv'
gdown.download(url, output, quiet=False)

# Load the dataframe
df = pd.read_csv(output, index_col=0)

print(f"DataFrame loaded successfully! Shape: {df.shape}")

Mounted at /content/drive


In [ ]:
df = df[df['Year'] == 2021]
df.shape

(3210, 24)

In [ ]:
df.head(1)

,Document,Dominant Topic,Labels,Topic_Concat,Topic,Contribution %,Topic Desc,Title,Author,Author_Count,...,CitedReference,Year,Interval,ResearchArea,Journal,CitedCount,AuthorAffiliation,CorrespondingAuthorAffiliation,Abstract_Title_Keywords,Page_Number
3151,3151,9,urban-rural development,T9-urban-rural development,T9-urban-rural development,27.18,"development, change, growth, economic, rural, ...",EXTENDED URBANISATION AND THE SPATIALITIES OF ...,"CONNOLLY, CREIGHTON;KEIL, ROGER;ALI, S. HARRIS",3,...,103,2021,2019-2021,ENVIRONMENTAL SCIENCES & ECOLOGY; URBAN STUDIES,URBAN STUDIES,118,UNIV LINCOLN;YORK UNIV,NATL UNIV SINGAPORE,THIS PAPER ARGUES THAT CONTEMPORARY PROCESSES ...,19.0


# keywords and new trend in 2021

## detect the emerging keywords and the keywords that regain popularity

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import spacy
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Load your data (assuming a CSV file with 'year' and 'keywords' columns)
data = pd.read_csv('Master_Analysis.csv', index_col=0)

def preprocess_keywords(keywords):
    if pd.isna(keywords):
        return []
    processed = []
    for kw in keywords.split(';'):
        doc = nlp(kw.strip().lower())
        lemmatized = " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])
        processed.append(lemmatized)
    return processed

data['processed_keywords'] = data['KeywordsPlus'].apply(preprocess_keywords)
data['Year'] = data['Year'].astype(int)

# Split the data into two periods
data_A = data[(data['Year'] >= 1991) & (data['Year'] <= 2020)]
data_B = data[data['Year'] >= 2021]

def get_keyword_frequencies(data):
    all_keywords = [keyword for sublist in data['processed_keywords'] for keyword in sublist]
    return Counter(all_keywords)

freq_A = get_keyword_frequencies(data_A)
freq_B = get_keyword_frequencies(data_B)

common_keywords = set(freq_A.keys()).intersection(set(freq_B.keys()))
emerging_keywords = {keyword: freq for keyword, freq in freq_B.items() if keyword not in common_keywords}
regained_popularity_keywords = {keyword: freq for keyword, freq in freq_B.items() if keyword in common_keywords and freq_B[keyword] > freq_A[keyword]}

# Convert to DataFrame for visualization
emerging_keywords_df = pd.DataFrame(emerging_keywords.items(), columns=['keyword', 'frequency'])
emerging_keywords_df = emerging_keywords_df.sort_values(by='frequency', ascending=False)

regained_popularity_df = pd.DataFrame(regained_popularity_keywords.items(), columns=['keyword', 'frequency'])
regained_popularity_df = regained_popularity_df.sort_values(by='frequency', ascending=False)

# Plot the emerging keywords
plt.figure(figsize=(10, 6))
sns.barplot(x='frequency', y='keyword', data=emerging_keywords_df.head(20), palette='viridis')
plt.xlabel('Frequency')
plt.title('Emerging Keywords (2021 and later)')
plt.gca().invert_yaxis()
plt.show()

# Plot the common keywords that have regained popularity
plt.figure(figsize=(10, 6))
sns.barplot(x='frequency', y='keyword', data=regained_popularity_df.head(20), palette='plasma')
plt.xlabel('Frequency')
plt.title('Keywords Regaining Popularity (2021 and later)')
plt.gca().invert_yaxis()
plt.show()

## Emerging keywords detection

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import spacy
from sklearn.feature_extraction.text import CountVectorizer

# Load your data (assuming a CSV file with 'year' and 'keywords' columns)
data = pd.read_csv('Master_Analysis.csv', index_col =0)

In [ ]:
!pip install nltk
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nltk.download('stopwords')
nlp = spacy.load('en_core_web_lg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess keywords
def preprocess_keywords(keywords):
    if pd.isna(keywords):
        return [], [], [], []

    unigrams = []
    bigrams = []
    trigrams = []

    for kw in keywords.split(';'):
        doc = nlp(kw.strip().lower())

        # Lemmatize and filter out stop words and non-alphabetic tokens
        lemmatized = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

        # Create unigrams, bigrams, and trigrams
        unigrams.extend(lemmatized)
        bigrams.extend([" ".join(lemmatized[i:i+2]) for i in range(len(lemmatized)-1)])
        trigrams.extend([" ".join(lemmatized[i:i+3]) for i in range(len(lemmatized)-2)])

    # Combine all levels
    combination = unigrams + bigrams + trigrams

    return unigrams, bigrams, trigrams, combination

# Apply the preprocessing function and create new columns for each level
data[['unigrams', 'bigrams', 'trigrams', 'combination']] = pd.DataFrame(
    data['Abstract_Title_Keywords'].apply(preprocess_keywords).tolist(), index=data.index
)

print(data[['unigrams', 'bigrams', 'trigrams', 'combination']].head())

                                                unigrams  \
44030                  [subcenter, los, angeles, region]   
44145    [spatial, mismatch, hypothesis, evidence, show]   
43933                     [asymmetric, tax, competition]   
43843  [paper, study, tax, competition, region, tax, ...   
43901                     [commuting, paradox, evidence]   

                                                 bigrams  \
44030       [subcenter los, los angeles, angeles region]   
44145  [spatial mismatch, mismatch hypothesis, hypoth...   
43933                  [asymmetric tax, tax competition]   
43843  [paper study, study tax, tax competition, comp...   
43901              [commuting paradox, paradox evidence]   

                                                trigrams  \
44030        [subcenter los angeles, los angeles region]   
44145  [spatial mismatch hypothesis, mismatch hypothe...   
43933                       [asymmetric tax competition]   
43843  [paper study tax, study tax com

In [ ]:
data['Year'] = data['Year'].astype(int)

In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
# Install xlsxwriter
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
# Define a list of words to remain lowercase
lowercase_words = {"and", "of", "on", "in", "the", "to", "with", "a", "an", "for", "at", "by"}

# Function to capitalize keywords appropriately
def capitalize_keywords(keywords):
    return ' '.join([word.capitalize() if word not in lowercase_words else word for word in keywords.split()])

# List of words or phrases to filter out (case insensitive)
filter_words = ['elsevi', 'elsevier', 'right reserve', 'ltd right', 'elsevi science', 'paper', 'describe', 'ltd', 'academic press','inc','C academic','inc right']

# Function to filter out keywords containing any of the words in filter_words
def filter_keywords(keywords):
    # Remove keywords containing any of the words in filter_words (case insensitive)
    return [kw for kw in keywords if not any(word in kw.lower() for word in filter_words)]

# Function to calculate frequencies for a given column
def get_keyword_frequencies(data, column_name):
    # Flatten the list of keywords, remove empty strings, and capitalize appropriately
    all_keywords = [
        capitalize_keywords(keyword)
        for sublist in data[column_name]
        for keyword in filter_keywords(sublist)
        if keyword.strip()  # Remove empty or whitespace-only keywords
    ]
    return Counter(all_keywords)

# Function to convert frequencies to a DataFrame with standardized values
def freq_to_dataframe(frequencies, top_n=200):
    top_words = frequencies.most_common(top_n)
    df = pd.DataFrame(top_words, columns=['Word', 'Frequency'])
    max_frequency = df['Frequency'].max()
    # Calculate the standardized frequency (0-100)
    df['Standardized Frequency'] = (df['Frequency'] / max_frequency) * 100
    return df

# Create an Excel writer object
excel_writer = pd.ExcelWriter('Result/topic_keyword_frequencies_t2.xlsx', engine='xlsxwriter')

# Initialize counters for combined frequencies across all topics for each level
combined_freq_A = {'unigrams': Counter(), 'bigrams': Counter(), 'trigrams': Counter(), 'combination': Counter()}
combined_freq_B = {'unigrams': Counter(), 'bigrams': Counter(), 'trigrams': Counter(), 'combination': Counter()}
combined_freq_C = {'unigrams': Counter(), 'bigrams': Counter(), 'trigrams': Counter(), 'combination': Counter()}
combined_freq_D = {'unigrams': Counter(), 'bigrams': Counter(), 'trigrams': Counter(), 'combination': Counter()}
combined_freq_E = {'unigrams': Counter(), 'bigrams': Counter(), 'trigrams': Counter(), 'combination': Counter()}
combined_freq_F = {'unigrams': Counter(), 'bigrams': Counter(), 'trigrams': Counter(), 'combination': Counter()}

# Loop through each unique topic
for topic in data['Topic'].unique():
    print(topic)

    # Filter data by topic and time periods
    data_A = data[(data['Topic'] == topic) & (data['Year'] >= 1991) & (data['Year'] <= 2000)]
    data_B = data[(data['Topic'] == topic) & (data['Year'] >= 2001) & (data['Year'] <= 2005)]
    data_C = data[(data['Topic'] == topic) & (data['Year'] >= 2006) & (data['Year'] <= 2010)]
    data_D = data[(data['Topic'] == topic) & (data['Year'] >= 2011) & (data['Year'] <= 2015)]
    data_E = data[(data['Topic'] == topic) & (data['Year'] >= 2016) & (data['Year'] <= 2018)]
    data_F = data[(data['Topic'] == topic) & (data['Year'] >= 2019) & (data['Year'] <= 2021)]

    # Calculate keyword frequencies for each level and time period
    for level in ['unigrams', 'bigrams', 'trigrams', 'combination']:
        freq_A = get_keyword_frequencies(data_A, level)
        freq_B = get_keyword_frequencies(data_B, level)
        freq_C = get_keyword_frequencies(data_C, level)
        freq_D = get_keyword_frequencies(data_D, level)
        freq_E = get_keyword_frequencies(data_E, level)
        freq_F = get_keyword_frequencies(data_F, level)

        # Update combined frequencies for each level
        combined_freq_A[level].update(freq_A)
        combined_freq_B[level].update(freq_B)
        combined_freq_C[level].update(freq_C)
        combined_freq_D[level].update(freq_D)
        combined_freq_E[level].update(freq_E)
        combined_freq_F[level].update(freq_F)

        # Convert frequencies to DataFrames with standardized values
        df_A = freq_to_dataframe(freq_A)
        df_B = freq_to_dataframe(freq_B)
        df_C = freq_to_dataframe(freq_C)
        df_D = freq_to_dataframe(freq_D)
        df_E = freq_to_dataframe(freq_E)
        df_F = freq_to_dataframe(freq_F)

        # Write each DataFrame to a separate sheet in the Excel file
        df_A.to_excel(excel_writer, sheet_name=f'{topic.split("-")[0]}_{level}_1991-2000', index=False)
        df_B.to_excel(excel_writer, sheet_name=f'{topic.split("-")[0]}_{level}_2001-2005', index=False)
        df_C.to_excel(excel_writer, sheet_name=f'{topic.split("-")[0]}_{level}_2006-2010', index=False)
        df_D.to_excel(excel_writer, sheet_name=f'{topic.split("-")[0]}_{level}_2011-2015', index=False)
        df_E.to_excel(excel_writer, sheet_name=f'{topic.split("-")[0]}_{level}_2016-2018', index=False)
        df_F.to_excel(excel_writer, sheet_name=f'{topic.split("-")[0]}_{level}_2019-2021', index=False)

# Convert combined frequencies to DataFrames with standardized values and write to Excel
for level in ['unigrams', 'bigrams', 'trigrams', 'combination']:
    combined_df_A = freq_to_dataframe(combined_freq_A[level])
    combined_df_B = freq_to_dataframe(combined_freq_B[level])
    combined_df_C = freq_to_dataframe(combined_freq_C[level])
    combined_df_D = freq_to_dataframe(combined_freq_D[level])
    combined_df_E = freq_to_dataframe(combined_freq_E[level])
    combined_df_F = freq_to_dataframe(combined_freq_F[level])

    # Write combined DataFrames to separate sheets in the Excel file
    combined_df_A.to_excel(excel_writer, sheet_name=f'Combined_{level}_1991-2000', index=False)
    combined_df_B.to_excel(excel_writer, sheet_name=f'Combined_{level}_2001-2005', index=False)
    combined_df_C.to_excel(excel_writer, sheet_name=f'Combined_{level}_2006-2010', index=False)
    combined_df_D.to_excel(excel_writer, sheet_name=f'Combined_{level}_2011-2015', index=False)
    combined_df_E.to_excel(excel_writer, sheet_name=f'Combined_{level}_2016-2018', index=False)
    combined_df_F.to_excel(excel_writer, sheet_name=f'Combined_{level}_2019-2021', index=False)

# Close the Excel file (saves and closes the writer)
excel_writer.close()
print("Keyword frequencies have been saved")

T6-Spatial effect
T5-Transportation
T2-Regional economics
T3-Socio-political geography
T8-Environmental management
T11-Public space and urban design
T4-Housing and property market
T7-Landscape and forestry
T1-Spatial analysis and modelling
T12-Planning policy and community governance
T10-Neighbourhood planning
T9-urban-rural development
Keyword frequencies have been saved


In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Function to generate a word cloud
def generate_wordcloud(df, sheet_name):
    # Create a dictionary of word frequencies
    word_freq = dict(zip(df['Word'], df['Standardized Frequency']))

    # Generate the word cloud with the shortest possible margin
    wordcloud = WordCloud(width=800, height=400, background_color='white', margin=0).generate_from_frequencies(word_freq)

    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    #plt.title(f'Word Cloud for {sheet_name}')

    # Save the word cloud as an image
    plt.savefig(f'Fig/Wordcloud/{sheet_name}.png', bbox_inches='tight')  # Save without extra white space
    plt.close()

# Read the Excel file
excel_file = 'Result/topic_keyword_frequencies_t2.xlsx'
xl = pd.ExcelFile(excel_file)

# Generate word clouds for each sheet
for sheet_name in tqdm(xl.sheet_names, desc="Generating Word Clouds"):
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    generate_wordcloud(df, sheet_name)

print("Word clouds have been generated for all sheets.")

Generating Word Clouds: 100%|██████████| 312/312 [15:21<00:00,  2.95s/it]

Word clouds have been generated for all sheets.
